In [ ]:
import numpy as np
from astropy.table import Table, vstack
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy import units as u

# TF fuji data

In [42]:
tf_fuji = Table.read('/global/cfs/projectdirs/desi/science/td/pv/desi_pv_tf_fuji_healpix.fits', format='fits')
tf_fuji[:5]

TARGETID,TARGET_RA,TARGET_DEC,HEALPIX,SURVEY,Z,ZERR,ZWARN,DELTACHI2,FILENAME,PVTYPE,SGA_ID,RA,DEC
int64,float64,float64,int64,bytes3,float64,float64,int64,float64,bytes63,bytes3,int64,float64,float64
1079550234591232,194.390863195343,27.5157211790145,10378,sv3,1.1235686466514,7.31685779475115e-05,4,3.28414569795132,fuji/healpix/sv3/bright/103/10378/redrock-sv3-bright-10378.fits,EXT,662902,194.39086319534337,27.51572117901454
1092744374124544,194.390863195343,27.5157211790145,10378,sv3,0.686773088332363,6.9756676262104e-05,4,0.786607094109058,fuji/healpix/sv3/dark/103/10378/redrock-sv3-dark-10378.fits,EXT,662902,194.39086319534337,27.51572117901454
1092744374124546,194.364461113654,27.5037185881314,10378,sv3,0.0242933923052181,4.95233472646785e-05,0,95.428411073226,fuji/healpix/sv3/dark/103/10378/redrock-sv3-dark-10378.fits,EXT,662902,194.36446111365385,27.50371858813136
1092744369930240,194.338458724402,27.4918902690326,10378,sv3,0.0264170223697961,0.00010139452689994,0,9.53278421035066,fuji/healpix/sv3/dark/103/10378/redrock-sv3-dark-10378.fits,EXT,662902,194.33845872440244,27.491890269032595
1092744374124545,194.377858465028,27.5098100780282,10378,sv3,0.211332646769145,6.68535116703737e-05,4,3.73989077657461,fuji/healpix/sv3/dark/103/10378/redrock-sv3-dark-10378.fits,EXT,662902,194.3778584650283,27.509810078028195


# TF iron data

In [43]:
tf_iron = Table.read('/global/cfs/projectdirs/desi/science/td/pv/desi_pv_tf_iron_healpix.fits', format='fits')
tf_iron[:5]

TARGETID,TARGET_RA,TARGET_DEC,HEALPIX,SURVEY,Z,ZERR,ZWARN,DELTACHI2,FILENAME,PVTYPE,SGA_ID,RA,DEC
int64,float64,float64,int64,bytes4,float64,float64,int64,float64,bytes65,bytes3,int64,float64,float64
2852147603439621,198.369130660983,36.5372037049171,10475,main,0.815976335547845,7.38513168100107e-05,4,0.128754377365112,iron/healpix/main/dark/104/10475/redrock-main-dark-10475.fits,EXT,649377,198.36913066098333,36.537203704917076
2399148812795907,198.371733180003,36.4994335406917,10475,main,1.11088784970434,7.48767797671894e-05,4,7.9473560154438,iron/healpix/main/bright/104/10475/redrock-main-bright-10475.fits,EXT,649377,198.37173318000336,36.499433540691676
2399382443917318,184.845242475328,49.8157304793777,10995,main,1.14739342108157,0.000146302276719084,4,2.56771463155746,iron/healpix/main/bright/109/10995/redrock-main-bright-10995.fits,EXT,1008911,184.84524247532795,49.81573047937771
2399634072797192,184.341289722203,70.8283725474297,11965,main,1.51703376230705,6.28979649962091e-05,4,4.76254060305655,iron/healpix/main/bright/119/11965/redrock-main-bright-11965.fits,EXT,241234,184.34128972220284,70.82837254742968
2852141710442505,123.256011148025,36.2652948002806,6448,main,0.00787379494184006,3.4714052819995e-05,0,22.1719104201402,iron/healpix/main/dark/64/6448/redrock-main-dark-6448.fits,EXT,31591,123.25601114802525,36.26529480028061


# TF Guadalupe data

In [44]:
tf_guadalupe = Table.read('/global/cfs/projectdirs/desi/science/td/pv/desi_pv_tf_guadalupe_healpix.fits', format='fits')
tf_guadalupe[:5]

TARGETID,TARGET_RA,TARGET_DEC,HEALPIX,SURVEY,Z,ZERR,ZWARN,DELTACHI2,FILENAME,PVTYPE,SGA_ID,RA,DEC
int64,float64,float64,int64,bytes7,float64,float64,int64,float64,bytes76,bytes3,int64,float64,float64
2399124498415623,214.013122385853,35.3145314097487,10511,main,1.377580664732,9.84449640590381e-05,4,5.34532008320093,guadalupe/healpix/main/bright/105/10511/redrock-main-bright-10511.fits,EXT,1229423,214.01312238585328,35.3145314097487
2412318637948931,214.04164239659,35.3585395889941,10511,main,0.0136149298283923,3.84349317724892e-05,0,105.808545001011,guadalupe/healpix/main/dark/105/10511/redrock-main-dark-10511.fits,EXT,1229423,214.04164239658957,35.35853958899406
2412333884243973,249.490218609158,36.058332514444,9433,main,0.790966031999166,5.68528719021782e-05,4,6.58435443043709,guadalupe/healpix/main/dark/94/9433/redrock-main-dark-9433.fits,EXT,306417,249.4902186091578,36.058332514444025
2412428541296646,246.94530009636,40.9339918585217,9471,main,1.59672805899643,0.00021512202669222,0,21.9079817682505,guadalupe/healpix/main/dark/94/9471/redrock-main-dark-9471.fits,EXT,1141359,246.94530009636003,40.93399185852167
2411343818784770,207.304942983762,-7.21204274377769,25721,main,0.0251758506722491,5.63666872037601e-05,0,18.4876595666283,guadalupe/healpix/main/dark/257/25721/redrock-main-dark-25721.fits,EXT,528682,207.30494298376212,-7.212042743777691


# TF master table
combining all data releases with flag ['DR'] </br>
1: FUJI</br>
2: IRON</br>
3: GUADALUPE

In [45]:
tf_fuji['DR'] = 1
tf_iron['DR'] = 2
tf_guadalupe['DR'] = 3

tf_master_table = vstack([tf_fuji, tf_iron, tf_guadalupe])

tf_master_table[:5]

TARGETID,TARGET_RA,TARGET_DEC,HEALPIX,SURVEY,Z,ZERR,ZWARN,DELTACHI2,FILENAME,PVTYPE,SGA_ID,RA,DEC,DR
int64,float64,float64,int64,bytes7,float64,float64,int64,float64,bytes76,bytes3,int64,float64,float64,int64
1079550234591232,194.390863195343,27.5157211790145,10378,sv3,1.1235686466514,7.31685779475115e-05,4,3.28414569795132,fuji/healpix/sv3/bright/103/10378/redrock-sv3-bright-10378.fits,EXT,662902,194.39086319534337,27.51572117901454,1
1092744374124544,194.390863195343,27.5157211790145,10378,sv3,0.686773088332363,6.9756676262104e-05,4,0.786607094109058,fuji/healpix/sv3/dark/103/10378/redrock-sv3-dark-10378.fits,EXT,662902,194.39086319534337,27.51572117901454,1
1092744374124546,194.364461113654,27.5037185881314,10378,sv3,0.0242933923052181,4.95233472646785e-05,0,95.428411073226,fuji/healpix/sv3/dark/103/10378/redrock-sv3-dark-10378.fits,EXT,662902,194.36446111365385,27.50371858813136,1
1092744369930240,194.338458724402,27.4918902690326,10378,sv3,0.0264170223697961,0.00010139452689994,0,9.53278421035066,fuji/healpix/sv3/dark/103/10378/redrock-sv3-dark-10378.fits,EXT,662902,194.33845872440244,27.491890269032595,1
1092744374124545,194.377858465028,27.5098100780282,10378,sv3,0.211332646769145,6.68535116703737e-05,4,3.73989077657461,fuji/healpix/sv3/dark/103/10378/redrock-sv3-dark-10378.fits,EXT,662902,194.3778584650283,27.509810078028195,1


In [46]:
tf_master_table.write('/pscratch/sd/n/nravi/desi_pv_tf_master_healpix.fits', format='fits', overwrite=True)

# SGA table

In [47]:
SGA = Table.read('/global/cfs/cdirs/cosmo/data/sga/2020/SGA-2020.fits', 'ELLIPSE')
SGA[:5]

SGA_ID,SGA_GALAXY,GALAXY,PGC,RA_LEDA,DEC_LEDA,MORPHTYPE,PA_LEDA,D25_LEDA,BA_LEDA,Z_LEDA,SB_D25_LEDA,MAG_LEDA,BYHAND,REF,GROUP_ID,GROUP_NAME,GROUP_MULT,GROUP_PRIMARY,GROUP_RA,GROUP_DEC,GROUP_DIAMETER,BRICKNAME,RA,DEC,D26,D26_REF,PA,BA,RA_MOMENT,DEC_MOMENT,SMA_MOMENT,G_SMA50,R_SMA50,Z_SMA50,SMA_SB22,SMA_SB22.5,SMA_SB23,SMA_SB23.5,SMA_SB24,SMA_SB24.5,SMA_SB25,SMA_SB25.5,SMA_SB26,G_MAG_SB22,R_MAG_SB22,Z_MAG_SB22,G_MAG_SB22.5,R_MAG_SB22.5,Z_MAG_SB22.5,G_MAG_SB23,R_MAG_SB23,Z_MAG_SB23,G_MAG_SB23.5,R_MAG_SB23.5,Z_MAG_SB23.5,G_MAG_SB24,R_MAG_SB24,Z_MAG_SB24,G_MAG_SB24.5,R_MAG_SB24.5,Z_MAG_SB24.5,G_MAG_SB25,R_MAG_SB25,Z_MAG_SB25,G_MAG_SB25.5,R_MAG_SB25.5,Z_MAG_SB25.5,G_MAG_SB26,R_MAG_SB26,Z_MAG_SB26,SMA_SB22_ERR,SMA_SB22.5_ERR,SMA_SB23_ERR,SMA_SB23.5_ERR,SMA_SB24_ERR,SMA_SB24.5_ERR,SMA_SB25_ERR,SMA_SB25.5_ERR,SMA_SB26_ERR,G_MAG_SB22_ERR,R_MAG_SB22_ERR,Z_MAG_SB22_ERR,G_MAG_SB22.5_ERR,R_MAG_SB22.5_ERR,Z_MAG_SB22.5_ERR,G_MAG_SB23_ERR,R_MAG_SB23_ERR,Z_MAG_SB23_ERR,G_MAG_SB23.5_ERR,R_MAG_SB23.5_ERR,Z_MAG_SB23.5_ERR,G_MAG_SB24_ERR,R_MAG_SB24_ERR,Z_MAG_SB24_ERR,G_MAG_SB24.5_ERR,R_MAG_SB24.5_ERR,Z_MAG_SB24.5_ERR,G_MAG_SB25_ERR,R_MAG_SB25_ERR,Z_MAG_SB25_ERR,G_MAG_SB25.5_ERR,R_MAG_SB25.5_ERR,Z_MAG_SB25.5_ERR,G_MAG_SB26_ERR,R_MAG_SB26_ERR,Z_MAG_SB26_ERR,G_COG_PARAMS_MTOT,G_COG_PARAMS_M0,G_COG_PARAMS_ALPHA1,G_COG_PARAMS_ALPHA2,G_COG_PARAMS_CHI2,R_COG_PARAMS_MTOT,R_COG_PARAMS_M0,R_COG_PARAMS_ALPHA1,R_COG_PARAMS_ALPHA2,R_COG_PARAMS_CHI2,Z_COG_PARAMS_MTOT,Z_COG_PARAMS_M0,Z_COG_PARAMS_ALPHA1,Z_COG_PARAMS_ALPHA2,Z_COG_PARAMS_CHI2,ELLIPSEBIT
int64,bytes16,bytes29,int64,float64,float64,bytes21,float32,float32,float32,float32,float32,float32,bool,bytes13,int64,bytes35,int16,bool,float64,float64,float32,bytes8,float64,float64,float32,bytes4,float32,float32,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32
2,SGA-2020 2,PGC1283207,1283207,228.3770865,5.4232017,S?,152.2,0.36307806,0.724436,0.03463229,23.40448,16.976,False,LEDA-20181114,0,PGC1283207,1,True,228.3770865,5.4232017,0.36307806,2283p055,228.3770803831908,5.423191398593787,0.49470574,SB26,158.20142,0.545691,228.37700918822188,5.4232652570544015,10.897086,3.3509698,3.1147978,3.240862,5.902337,6.9126143,7.941369,8.997992,10.073601,11.199986,12.391357,13.561038,14.841172,16.966799,16.108246,15.486356,16.879545,16.024958,15.400715,16.818878,15.967034,15.341793,16.776297,15.925804,15.300776,16.746685,15.897334,15.272053,16.725166,15.876816,15.2521105,16.708357,15.862035,15.237181,16.696539,15.851936,15.226998,16.689613,15.844313,15.21976,0.013392451,0.02354,0.021872982,0.01736985,0.024445537,0.039866067,0.05026544,0.08455789,0.122911856,0.005682776,0.0054258136,0.0049038026,0.005588406,0.005323561,0.0047632363,0.00543534,0.005177031,0.0046343105,0.0053025587,0.005040888,0.0045181247,0.005206092,0.0049438984,0.0044374703,0.0051483097,0.0048758644,0.0043834248,0.0051032505,0.0048264163,0.004344248,0.0050705094,0.004792021,0.004319857,0.005054293,0.004765629,0.0043044444,16.65942,0.34037337,0.2978292,3.0239506,0.07928849,15.820566,0.2640441,0.34559453,3.3033552,0.003811298,15.195567,0.29826432,0.3001073,3.2333765,0.011723555,0
3,SGA-2020 3,PGC1310416,1310416,202.54443750000002,6.9345944,Sc,159.26,0.4017908,0.7816278,0.073888786,23.498482,16.85,False,LEDA-20181114,1,PGC1310416,1,True,202.54443750000002,6.9345944,0.4017908,

In [48]:
SGA_dict={}
for i in range(len(SGA)):
    SGA_dict[SGA['SGA_ID'][i]] = i

# center vs. off-center observations

In [49]:
tf_master_table['SKY_FIBER_DIST'] = 0.
tf_master_table['SKY_FIBER_DIST_R26'] = 0.

# For each SGA galaxy that has at least one observation, calculate the distance 
# for all of that galaxy's targets
for sga_id in np.unique(tf_master_table['SGA_ID']):
    
    # Identify all galaxy targets on this galaxy
    obs_idx = tf_master_table['SGA_ID'] == sga_id
    
    # Find galaxy index in SGA catalog
    sga_idx = SGA_dict[sga_id]
    
    #---------------------------------------------------------------------------
    # Calculate distance between each observation and the center of the SGA 
    # galaxy
    #---------------------------------------------------------------------------
    SGA_coords = SkyCoord(ra=SGA['RA'][sga_idx], 
                          dec=SGA['DEC'][sga_idx], 
                          unit=u.degree)
    target_coords = SkyCoord(ra=tf_master_table['RA'][obs_idx], 
                             dec=tf_master_table['DEC'][obs_idx], 
                             unit=u.degree)
    
    sep2d = target_coords.separation(SGA_coords)
    
    tf_master_table['SKY_FIBER_DIST'][obs_idx] = sep2d
    tf_master_table['SKY_FIBER_DIST_R26'][obs_idx] = 2*sep2d.to('arcmin')/(SGA['D26'][sga_idx]*u.arcmin)

In [10]:
tf_ext = tf_master_table[tf_master_table['PVTYPE']=='EXT']

# Sort EXT observations
want EOA flag, want to VI EOA galaxies to make sure only fibers on galaxy are included

In [11]:
tf_ext_sga_ids = np.unique(tf_ext['SGA_ID'])
tf_master_table['EXT'] = 0.

# flag all observations belonging to EXT galaxies

for i in range(len(tf_master_table)):
    if tf_master_table['SGA_ID'][i] in tf_ext_sga_ids:
        tf_master_table['EXT'][i] = 1
        

In [12]:
off_axis = Table.read('/global/homes/n/nravi/SGA_off-axis_targets.fits', format='fits')
off_axis[:5]

GALAXY,RA,DEC
bytes11,float64,float64
NGC4761,193.29086531640286,-9.197860127650477
NGC4761,193.25307026043834,-9.187776242212069
NGC4761,193.26884118382742,-9.20154030566786
NGC4761,193.26838380153842,-9.198178779994157
NGC4761,193.27011185413699,-9.199483271700393


In [23]:
ra_dec_dict = {}
for i in range(len(tf_master_table)):
    ra_dec_dict[('%.8f'%(tf_master_table['TARGET_RA'][i]),'%.8f'%(tf_master_table['TARGET_DEC'][i]))] = i

In [24]:
tf_master_table['EOA'] = 0

In [25]:
tf_master_table[:5]

TARGETID,TARGET_RA,TARGET_DEC,HEALPIX,SURVEY,Z,ZERR,ZWARN,DELTACHI2,FILENAME,PVTYPE,SGA_ID,RA,DEC,DR,SKY_FIBER_DIST,SKY_FIBER_DIST_R26,EXT,EOA
int64,float64,float64,int64,bytes7,float64,float64,int64,float64,bytes76,bytes3,int64,float64,float64,int64,float64,float64,float64,int64
1079550234591232,194.390863195343,27.5157211790145,10378,sv3,1.1235686466514,7.31685779475115e-05,4,3.28414569795132,fuji/healpix/sv3/bright/103/10378/redrock-sv3-bright-10378.fits,EXT,662902,194.39086319534337,27.51572117901454,1,0.03927420444753712,1.0000000170652443,1.0,0
1092744374124544,194.390863195343,27.5157211790145,10378,sv3,0.686773088332363,6.9756676262104e-05,4,0.786607094109058,fuji/healpix/sv3/dark/103/10378/redrock-sv3-dark-10378.fits,EXT,662902,194.39086319534337,27.51572117901454,1,0.03927420444753712,1.0000000170652443,1.0,0
1092744374124546,194.364461113654,27.5037185881314,10378,sv3,0.0242933923052181,4.95233472646785e-05,0,95.428411073226,fuji/healpix/sv3/dark/103/10378/redrock-sv3-dark-10378.fits,EXT,662902,194.36446111365385,27.50371858813136,1,0.012960487467698754,0.3300000056318236,1.0,0
1092744369930240,194.338458724402,27.4918902690326,10378,sv3,0.0264170223697961,0.00010139452689994,0,9.53278421035066,fuji/healpix/sv3/dark/103/10378/redrock-sv3-dark-10378.fits,EXT,662902,194.33845872440244,27.491890269032595,1,0.012960487467685656,0.33000000563149007,1.0,0
1092744374124545,194.377858465028,27.5098100780282,10378,sv3,0.211332646769145,6.68535116703737e-05,4,3.73989077657461,fuji/healpix/sv3/dark/103/10378/redrock-sv3-dark-10378.fits,EXT,662902,194.3778584650283,27.509810078028195,1,0.026313716979861984,0.6700000114340222,1.0,0


In [26]:
for i in range(len(off_axis)):
    try:
        i_m = ra_dec_dict[('%.8f'%(off_axis['RA'][i]), '%.8f'%(off_axis['DEC'][i]))]
    except:
        #print(i)
        continue
    
    tf_master_table['EOA'][i_m] = 1

In [27]:
tf_master_table[tf_master_table['EOA'] > 0]

TARGETID,TARGET_RA,TARGET_DEC,HEALPIX,SURVEY,Z,ZERR,ZWARN,DELTACHI2,FILENAME,PVTYPE,SGA_ID,RA,DEC,DR,SKY_FIBER_DIST,SKY_FIBER_DIST_R26,EXT,EOA
int64,float64,float64,int64,bytes7,float64,float64,int64,float64,bytes76,bytes3,int64,float64,float64,int64,float64,float64,float64,int64


In [29]:
tf_master_table.write('/pscratch/sd/n/nravi/desi_pv_tf_master_healpix.fits', format='fits', overwrite=True)

In [30]:
tf_ext_targets = tf_master_table[tf_master_table['EXT']==1]
tf_ext_sga = tf_ext_targets[np.logical_and(tf_ext_targets['TARGETID'] > 30000000000000000, tf_ext_targets['TARGETID'] < 40000000000000000)]

In [31]:
a, b = np.unique(tf_ext_targets['SGA_ID'], return_counts=True)
np.where(tf_ext_sga['SGA_ID'] == a[48])

(array([ 19, 122]),)

In [32]:
tf_ext_sga[:5]

TARGETID,TARGET_RA,TARGET_DEC,HEALPIX,SURVEY,Z,ZERR,ZWARN,DELTACHI2,FILENAME,PVTYPE,SGA_ID,RA,DEC,DR,SKY_FIBER_DIST,SKY_FIBER_DIST_R26,EXT,EOA
int64,float64,float64,int64,bytes7,float64,float64,int64,float64,bytes76,bytes3,int64,float64,float64,int64,float64,float64,float64,int64
39633062551620033,153.178857933669,39.3739445564769,5703,sv1,0.022711924294578,5.75568615760296e-07,0,641871.757860661,fuji/healpix/sv1/bright/57/5703/redrock-sv1-bright-5703.fits,EXT,1037949,153.17885793366855,39.3739445564769,1,0.0,0.0,1.0,0
39633062056692288,115.163713024568,39.2330431988692,6560,sv1,0.0112289665628835,2.5580485409679e-06,0,199314.054352144,fuji/healpix/sv1/bright/65/6560/redrock-sv1-bright-6560.fits,EXT,17643,115.16371302456768,39.2330431988692,1,1.1007441864020942e-14,2.0732856604950621e-13,1.0,0
39627865758633464,150.11281896177,3.37442274219938,27345,sv3,0.00687190356978907,1.5141461957797e-06,0,24405.1790848412,fuji/healpix/sv3/bright/273/27345/redrock-sv3-bright-27345.fits,EXT,1225398,150.1128189617695,3.3744227421993793,1,0.0,0.0,1.0,0
39627770296276005,219.956999318774,-0.718640392418429,25933,sv3,0.00585538786570846,4.19501768104605e-06,0,2858.02824933385,fuji/healpix/sv3/bright/259/25933/redrock-sv3-bright-25933.fits,EXT,166290,219.95699931877425,-0.7186403924184287,1,0.0,0.0,1.0,0
39628422518932597,193.522783733207,27.1486963667462,28149,sv3,0.00127843533483607,1.61464939000045e-05,0,43.380185824275,fuji/healpix/sv3/bright/281/28149/redrock-sv3-bright-28149.fits,TFT,940344,193.52278373320706,27.148696366746147,1,0.0,0.0,1.0,0


In [33]:
# test just one galaxy
from inspect_cutouts import *



No module named 'plot_funcs'
/pscratch/sd/n/nravi/tf_ext_vi_cache/39627776327685463.jpg exists.


In [41]:
# collect all observations for this galaxy
for i in range(len(a)):
    sga_id = a[i]
    idx = b[i]

    tf_ext_sga['TARGETID'][idx]
    sga_id = tf_ext_sga['SGA_ID'][idx]
    d26 = float(SGA['D26'][SGA['SGA_ID'] == sga_id][0])
    npix = np.minimum(int(1.025 * d26*60/0.262), 512)


    img_file, wcs = get_cutout(tf_ext_sga['TARGETID'][idx], tf_ext_sga['TARGET_RA'][idx], tf_ext_sga['TARGET_DEC'][idx], size=npix, verbose=True)
    img = mpl.image.imread(img_file)


    fibers = tf_ext_targets[tf_ext_targets['SGA_ID'] == sga_id]

    fig1 = plt.figure(figsize=(7,7))

    ax = fig1.add_subplot(111, projection=wcs)
    ax.imshow(np.flip(img, axis=0))
    ax.set(xlabel='ra', ylabel='dec')
    ax.text(int(0.02*npix), int(0.85*npix), 'TARGETID: {}\nSGA_ID: {}\n$z={{{:.4f}}}$'.format(tf_ext_sga['TARGETID'][19], sga_id, tf_ext_sga['Z'][19]), fontsize=9, color='yellow')
    overlay = ax.get_coords_overlay('icrs')
    overlay.grid(color='white', ls='dotted');

    for f in fibers:
    
        ra = f['TARGET_RA']
        dec = f['TARGET_DEC']
        targetid = f['TARGETID']
    
        r1 = SphericalCircle((ra * u.deg, dec * u.deg), 2 * u.arcsec,
                         edgecolor='m',
                         facecolor='none', alpha=0.8, lw=3,
                         transform=ax.get_transform('icrs'),
                        #    label=targetid
                        )
    
        ax.add_patch(r1)

# plot fiber locations with targetid labeled
#ax.legend(loc='center left', 
#          bbox_to_anchor=(1.2, 0.5)
#         )
#plt.title(sga_id)
    plt.savefig('/pscratch/sd/n/nravi/tf_ext_vi_cache/'+ str(sga_id) + '.jpg')
    plt.close()

Get https://www.legacysurvey.org/viewer/cutout.jpg?ra=150.11281896177&dec=3.37442274219938&zoom=14&layer=ls-dr9&size=512&sga
/pscratch/sd/n/nravi/tf_ext_vi_cache/39627865758633464.jpg exists.
Get https://www.legacysurvey.org/viewer/cutout.jpg?ra=179.789097798572&dec=52.7075991081414&zoom=14&layer=ls-dr9&size=512&sga
/pscratch/sd/n/nravi/tf_ext_vi_cache/39627865758633464.jpg exists.
/pscratch/sd/n/nravi/tf_ext_vi_cache/39627865758633464.jpg exists.
/pscratch/sd/n/nravi/tf_ext_vi_cache/39633062056692288.jpg exists.
Get https://www.legacysurvey.org/viewer/cutout.jpg?ra=219.956999318774&dec=-0.718640392418429&zoom=14&layer=ls-dr9&size=512&sga
Get https://www.legacysurvey.org/viewer/cutout.jpg?ra=208.909454506112&dec=4.98431229878542&zoom=14&layer=ls-dr9&size=512&sga
/pscratch/sd/n/nravi/tf_ext_vi_cache/39627865758633464.jpg exists.
/pscratch/sd/n/nravi/tf_ext_vi_cache/39627770296276005.jpg exists.
/pscratch/sd/n/nravi/tf_ext_vi_cache/39633062056692288.jpg exists.
/pscratch/sd/n/nravi/tf_ex

In [60]:
fibers

TARGETID,TARGET_RA,TARGET_DEC,HEALPIX,SURVEY,Z,ZERR,ZWARN,DELTACHI2,FILENAME,PVTYPE,SGA_ID,RA,DEC,DR,SKY_FIBER_DIST,SKY_FIBER_DIST_R26,EXT
int64,float64,float64,int64,bytes7,float64,float64,int64,float64,bytes76,bytes3,int64,float64,float64,int64,float64,float64,float64
1083296704364545,219.45730672238,-0.394088735219175,25935,sv3,0.457998928200844,9.41339911156193e-05,4,0.00438238307833672,fuji/healpix/sv3/dark/259/25935/redrock-sv3-dark-25935.fits,EXT,136307,219.45730672238003,-0.3940887352191747,1,0.016227048786034346,0.670000016880562,1.0
1083296704364546,219.465197462072,-0.396444193999683,25935,sv3,-0.00199569129234795,4.13114935733491e-48,1570,1.94266889222573e+84,fuji/healpix/sv3/dark/259/25935/redrock-sv3-dark-25935.fits,EXT,136307,219.46519746207196,-0.39644419399968267,1,0.007992427014007568,0.3300000083139206,1.0
1083296704364547,219.480514793136,-0.401016533657631,25935,sv3,0.00606096720561597,2.17841019625403e-05,0,482.104535131848,fuji/healpix/sv3/dark/259/25935/redrock-sv3-dark-25935.fits,EXT,136307,219.48051479313645,-0.4010165336576305,1,0.007992427014035524,0.330000008315075,1.0
1070102564831236,219.488405546108,-0.403371970195963,25935,sv3,0.480299270325962,6.26832077621877e-05,0,13.1833695992827,fuji/healpix/sv3/bright/259/25935/redrock-sv3-bright-25935.fits,EXT,136307,219.48840554610794,-0.4033719701959629,1,0.01622704878606321,0.6700000168817537,1.0
1083296704364548,219.488405546108,-0.403371970195963,25935,sv3,0.497060791424862,7.17312091637611e-05,0,20.0267319288105,fuji/healpix/sv3/dark/259/25935/redrock-sv3-dark-25935.fits,EXT,136307,219.48840554610794,-0.4033719701959629,1,0.01622704878606321,0.6700000168817537,1.0
1083296704364545,219.45730672238,-0.394088735219175,25935,sv3,0.457998928200844,9.41339911156193e-05,4,0.00438238307833672,fuji/healpix/sv3/dark/259/25935/redrock-sv3-dark-25935.fits,TFT,136307,219.45730672238003,-0.3940887352191747,1,0.016227048786034346,0.670000016880562,1.0
1083296704364546,219.465197462072,-0.396444193999683,25935,sv3,-0.00199569129234795,4.13114935733491e-48,1570,1.94266889222573e+84,fuji/healpix/sv3/dark/259/25935/redrock-sv3-dark-25935.fits,TFT,136307,219.46519746207196,-0.39644419399968267,1,0.007992427014007568,0.3300000083139206,1.0
1083296704364547,219.480514793136,-0.401016533657631,25935,sv3,0.00606096720561597,2.17841019625403e-05,0,482.104535131848,fuji/healpix/sv3/dark/259/25935/redrock-sv3-dark-25935.fits,TFT,136307,219.48051479313645,-0.4010165336576305,1,0.007992427014035524,0.330000008315075,1.0
1070102564831236,219.488405546108,-0.403371970195963,25935,sv3,0.480299270325962,6.26832077621877e-05,0,13.1833695992827,fuji/healpix/sv3/bright/259/25935/redrock-sv3-bright-25935.fits,TFT,136307,219.48840554610794,-0.4033719701959629,1,0.01622704878606321,0.6700000168817537,1.0


In [ ]:
for sga_id in dist_SGA_ids:
    galaxy_list = inComa_sga_table[inComa_sga_table['SGA_ID'] == sga_id]
    
    # Total hack to find SGA target, because the PVTYPE can't be trusted.
    # Talk to an expert about actual SGA targets in DESI.
    is_sga_galaxy = (galaxy_list['TARGETID'] > 30000000000000000) & (galaxy_list['TARGETID'] < 40000000000000000)
    
    sga_galaxy = galaxy_list[is_sga_galaxy]
    tf_list = galaxy_list[~is_sga_galaxy]
    '''
    if np.sum(is_sga_galaxy) < 1:
        print('No center observation of galaxy {}'.format(sga_id))
        continue
    ''';
    if np.sum(is_sga_galaxy) > 1:
        # Multiple observations of the center - just take the first one in the table
#         print('{} observations of galaxy center for {}'.format(np.sum(is_sga_galaxy), sga_id))
        sga_galaxy = sga_galaxy[0]
        '''
        print('Too many records for {}'.format(sga_id))
        continue
        ''';
    # print(np.dtype(sga_galaxy['TARGETID']))
    try:
        targetid = int(sga_galaxy['TARGETID'])
    except TypeError as err:
        print(err)
        # print(sga_galaxy['TARGETID'])
        continue
    ra, dec, z = float(sga_galaxy['FIBER_RA']), float(sga_galaxy['FIBER_DEC']), float(sga_galaxy['Z'])
    
    # D26 in arcmin
    # print(SGA['D26'][SGA['SGA_ID'] == sga_id][0])
    d26 = float(SGA['D26'][SGA['SGA_ID'] == sga_id][0])
    
    
    center = SkyCoord(sga_galaxy['FIBER_RA'], sga_galaxy['FIBER_DEC'], unit='deg')
    offcenter = SkyCoord(tf_list['FIBER_RA'], tf_list['FIBER_DEC'], unit='deg')
    sep2d = offcenter.separation(center)
    #idx, sep2d, sep3d = match_coordinates_sky(offcenter, center)
    '''
    isoffcenter = sep2d.to('arcsec') > 1.5*u.arcsec
    if np.sum(isoffcenter) == 0:
        print('No observations off the galactic center for {}'.format(sga_id))
        continue
    ''';
    # Padd the image cutout of the galaxy.
    # Multiply by 60 (to arcsec), divide by 180 to get pixscale.
#     pixscale = 1.05*d26*60/180
    npix = np.minimum(int(1.025 * d26*60/0.262), 512)
    
    #print(targetid, sga_id, ra, dec)
#     img_file = get_cutout(targetid, ra, dec, size=npix, verbose=True)
    img_file, wcs = get_cutout(targetid, ra, dec, size=npix, verbose=True)
    img = mpl.image.imread(img_file)

    fig1 = plt.figure(figsize=(7,5))

    ax = fig1.add_subplot(111, projection=wcs)
    ax.imshow(np.flip(img, axis=0))
    ax.set(xlabel='ra', ylabel='dec')
    ax.text(int(0.02*npix), int(0.85*npix), 'TARGETID: {}\nSGA_ID: {}\n$z={{{:.4f}}}$'.format(targetid, sga_id, z), fontsize=9, color='yellow')
    overlay = ax.get_coords_overlay('icrs')
    overlay.grid(color='white', ls='dotted');

    # Add the location of the SDSS and DESI fibers.
    # SDSS fibers are 2" diameter, DESI is 107 um with 70 um/" plate scale.
    r1 = SphericalCircle((ra * u.deg, dec * u.deg), 2 * u.arcsec,
                         edgecolor='black', facecolor='none', alpha=0.8, lw=3,
                         transform=ax.get_transform('icrs'))
    r2 = SphericalCircle((ra * u.deg, dec * u.deg), 2 * u.arcsec,
                         edgecolor='red', facecolor='none', alpha=0.8, lw=2,
                         transform=ax.get_transform('icrs'))
    ax.add_patch(r1)
    ax.add_patch(r2)

    for tft in tf_list:
        ra, dec = float(tft['FIBER_RA']), float(tft['FIBER_DEC'])
        
        edgecolor2 = 'orange'
#         if tft['Z'] > 0.05:
#             edgecolor2 = 'orange'
#         else:
#             edgecolor2 = 'lime'
        
        # Add the location of the SDSS and DESI fibers.
        # SDSS fibers are 2" diameter, DESI is 107 um with 70 um/" plate scale.
        r1 = SphericalCircle((ra * u.deg, dec * u.deg), 2 * u.arcsec,
                             edgecolor='lightcoral', facecolor='none', alpha=1, lw=3,
                             transform=ax.get_transform('icrs'))
        r2 = SphericalCircle((ra * u.deg, dec * u.deg), 2 * u.arcsec,
                             edgecolor=edgecolor2, facecolor='none', alpha=0.8, lw=2,
                             transform=ax.get_transform('icrs'))
        ax.add_patch(r1)
        ax.add_patch(r2)
    
    fig1.subplots_adjust(top=0.85, right=0.85, bottom=0.15, left=0.15)
    
#     coadds = get_spectra_for_sga(inComa_sga_table, sga_id)
#     n = coadds.num_spectra()

#     fig2, axes = plt.subplots(n,1, figsize=(8,4*n), sharex=True, sharey=True, tight_layout=True)

#     for i in range(n):
#         wave = coadds.wave['brz']
#         flux = coadds.flux['brz'][i]
#         smoothed = gaussian_filter1d(flux, 7)

#         ax = axes[i]
#         ax.plot(wave, smoothed, label='TARGETID: {}'.format(coadds.fibermap['TARGETID'][i]))
#         ax.set(ylabel=r'flux [$10^{-17}$ erg cm$^{-2}$ s$^{-1}$ $\AA^{-1}$]')
#         if i+1 == n:
#             ax.set(xlabel=r'$\lambda_\mathrm{obs}$ [$\AA$]')

#         ax.legend(loc='upper right', fontsize=10)
    
    # figure = PdfPages('cache/DM_cutouts/dist_{}.pdf'.format(sga_id))
    # figure.savefig(fig1, dpi = 300)
    # figure.savefig(fig2, dpi = 120)
    # figure.close()
    fig1.savefig('cache/DM_cutouts/dist_{}.png'.format(sga_id), dpi=120, overwrite = True)
#     break
    
#     fig.clear()
#     plt.close(fig)